In [2]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

In [5]:
# load in and keep only sandy transects
df_annual = pd.read_csv('00. data/westeurope_annual.csv')
df_annual = df_annual[df_annual['flag_sandy'] == True]


df_annual = gpd.GeoDataFrame(df_annual, geometry= gpd.points_from_xy(df_annual.Intersect_lon, df_annual.Intersect_lat))
df_annual.crs = {"init": "epsg:3587"}
df_annual = df_annual.to_crs({'init': 'epsg:32611'})

C:\Users\heijd_de\AppData\Roaming\Python\Python38\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


#### GHG emmisions (Vousdoukas, 2021)

In [37]:
basepath_vous = r'P:\1000545-054-globalbeaches\01_Global_Future_Shorelines\globalErosionProjectionsDataset_linear'

df_vous_45_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2050.csv'), header= None)
df_vous_45_2050 = df_vous_45_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2050', 3: 'p5_45_2050', 
                                                   4: 'p17_45_2050', 5: 'p50_45_2050', 6: 'p83_45_2050', 7: 
                                                   'p95_45_2050', 8: 'p99_45_2050'})
df_vous_45_2050 = df_vous_45_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')
# df_vous_45_2050 = gpd.GeoDataFrame(df_vous_45_2050, geometry= gpd.points_from_xy(df_vous_45_2050.longitude, 
#                                                                                  df_vous_45_2050.latitude))
# df_vous_45_2050.crs = {"init": "epsg:3587"}
# df_vous_45_2050 = df_vous_45_2050.to_crs({'init': 'epsg:32611'})


df_vous_45_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2100.csv'), header= None)
df_vous_45_2100 = df_vous_45_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2100', 3: 'p5_45_2100', 
                                                   4: 'p17_45_2100', 5: 'p50_45_2100', 6: 'p83_45_2100', 7: 
                                                   'p95_45_2100', 8: 'p99_45_2100'})
df_vous_45_2100 = df_vous_45_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')
# df_vous_45_2100 = gpd.GeoDataFrame(df_vous_45_2100, geometry= gpd.points_from_xy(df_vous_45_2100.longitude, 
#                                                                                  df_vous_45_2100.latitude))
# df_vous_45_2100.crs = {"init": "epsg:3587"}
# df_vous_45_2100 = df_vous_45_2100.to_crs({'init': 'epsg:32611'})

df_vous_85_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2050.csv'), header= None)
df_vous_85_2050 = df_vous_85_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2050', 3: 'p5_85_2050', 
                                                   4: 'p17_85_2050', 5: 'p50_85_2050', 6: 'p83_85_2050', 7: 
                                                   'p95_85_2050', 8: 'p99_85_2050'})
df_vous_85_2050 = df_vous_85_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')
# df_vous_85_2050 = gpd.GeoDataFrame(df_vous_85_2050, geometry= gpd.points_from_xy(df_vous_85_2050.longitude, 
#                                                                                  df_vous_85_2050.latitude))
# df_vous_85_2050.crs = {"init": "epsg:3587"}
# df_vous_85_2050 = df_vous_85_2050.to_crs({'init': 'epsg:32611'})


df_vous_85_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2100.csv'), header= None)
df_vous_85_2100 = df_vous_85_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2100', 3: 'p5_85_2100', 
                                                   4: 'p17_85_2100', 5: 'p50_85_2100', 6: 'p83_85_2100', 7: 
                                                   'p95_85_2100', 8: 'p99_85_2100'})
df_vous_85_2100 = df_vous_85_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')
# df_vous_85_2100 = gpd.GeoDataFrame(df_vous_85_2100, geometry= gpd.points_from_xy(df_vous_85_2100.longitude, 
#                                                                                  df_vous_85_2100.latitude))
# df_vous_85_2100.crs = {"init": "epsg:3587"}
# df_vous_85_2100 = df_vous_85_2100.to_crs({'init': 'epsg:32611'})

df_vous_45_2050.head(2)

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239


In [30]:
[set(df_vous_45_2050['latitude']).difference(x) for x in [set(df_vous_45_2100['latitude']), set(df_vous_85_2050['latitude']), set(df_vous_85_2100['latitude'])]]


[set(), set(), set()]

In [38]:
dfs_vous = [df_vous_45_2100, df_vous_85_2050, df_vous_85_2100]

df_vous = df_vous_45_2050.copy()
for dfv in dfs_vous:
    df_vous = df_vous.merge(dfv, on = ['latitude', 'longitude'], how = 'left')

print(len(df_vous_45_2050), len(df_vous))

504815 504815
504815 504815


In [ ]:
df_vous = gpd.GeoDataFrame(df_vous, geometry= gpd.points_from_xy(df_vous.longitude, 
                                                                 df_vous_85_2100.latitude))
df_vous_85_2100.crs = {"init": "epsg:3587"}
df_vous_85_2100 = df_vous_85_2100.to_crs({'init': 'epsg:32611'})

In [21]:
df_vous[duplicates]

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050,geometry,...,p83_85_2050,p95_85_2050,p99_85_2050,p1_85_2100,p5_85_2100,p17_85_2100,p50_85_2100,p83_85_2100,p95_85_2100,p99_85_2100
70,-53.307,-73.496,-18.305,-14.527,-10.651,-6.3262,-3.1673,-1.8965,-1.6001,POINT (-2378801.482 2811076.548),...,-3.7896,-2.1854,-1.6995,-100.42,-66.325,-46.326,-26.139,-12.943,-7.4884,-5.5639
71,-53.307,-73.496,-18.305,-14.527,-10.651,-6.3262,-3.1673,-1.8965,-1.6001,POINT (-2378801.482 2811076.548),...,-3.7896,-2.1854,-1.6995,-100.42,-66.325,-46.326,-26.139,-12.943,-7.4884,-5.5639
72,-53.307,-73.496,-18.305,-14.527,-10.651,-6.3262,-3.1673,-1.8965,-1.6001,POINT (-2378801.482 2811076.548),...,-3.7896,-2.1854,-1.6995,-100.42,-66.325,-46.326,-26.139,-12.943,-7.4884,-5.5639
73,-53.307,-73.496,-18.305,-14.527,-10.651,-6.3262,-3.1673,-1.8965,-1.6001,POINT (-2378801.482 2811076.548),...,-3.7896,-2.1854,-1.6995,-100.42,-66.325,-46.326,-26.139,-12.943,-7.4884,-5.5639
74,-53.307,-73.496,-18.305,-14.527,-10.651,-6.3262,-3.1673,-1.8965,-1.6001,POINT (-2378801.482 2811076.548),...,-3.7896,-2.1854,-1.6995,-100.42,-66.325,-46.326,-26.139,-12.943,-7.4884,-5.5639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806424,64.656,39.818,-81.706,-68.770,-55.462,-35.0100,-18.8910,-12.0700,-9.9639,POINT (-2378763.918 2811241.035),...,-40.7670,-24.4160,-18.9680,-672.22,-522.370,-351.660,-220.210,-112.110,-62.1000,-50.4350
806425,64.656,39.818,-81.706,-68.770,-55.462,-35.0100,-18.8910,-12.0700,-9.9639,POINT (-2378763.918 2811241.035),...,-40.7670,-24.4160,-18.9680,-672.22,-522.370,-351.660,-220.210,-112.110,-62.1000,-50.4350
806426,64.656,39.818,-81.706,-68.770,-55.462,-35.0100,-18.8910,-12.0700,-9.9639,POINT (-2378763.918 2811241.035),...,-40.7670,-24.4160,-18.9680,-672.22,-522.370,-351.660,-220.210,-112.110,-62.1000,-50.4350
806427,64.656,39.818,-81.706,-68.770,-55.462,-35.0100,-18.8910,-12.0700,-9.9639,POINT (-2378763.918 2811241.035),...,-40.7670,-24.4160,-18.9680,-672.22,-522.370,-351.660,-220.210,-112.110,-62.1000,-50.4350


In [ ]:
df_vous_tot = df_vous_tot.drop_duplicates(subset='transect_id', keep=False)

In [ ]:
df_merged3 = df_merged2.merge(df_vous_tot, on = 'transect_id', how = 'left')